# 1 Notebook Setup

In [1]:
import os
import sys
import platform
import subprocess
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
import pandas as pd
import pickle

In [2]:
# Add src and dependencies/Model_Checking folders to environmental path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'src')))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'dependencies','Model_Checking')))

In [3]:
# Build non-python dependencies for the usage of function runMarkovCluster()
# uncomment this cell only if running the notebook locally on MacOS/Linux and you haven't build it yet in setup.py
# otherwise, skip it

# command = "cd ../dependencies && tar -xvzf mcl-14-137.tar.gz && cd mcl-14-137 && "
# command += "./configure && make && make install && make clean && make distclean"
# process = subprocess.Popen(command, shell=True)
# process.wait()

In [4]:
# Build non-python dependencies for the usage of function call_checker()
# uncomment this cell only if running the notebook locally on MacOS/Linux and you haven't build it yet in setup.py

# command = "cd ../dependencies/gsl-2.7.1 && tar -xvzf gsl-2.7.1.tar.gz && cd gsl-2.7.1 && "
# command += "./configure && make && make install && make clean && make distclean"
# command += " && cd ../Model_Checking/dishwrap_v1.0/dishwrap && make"
# command += " && cd ../monitor && make"
# process = subprocess.Popen(command, shell=True)
# process.wait()

In [5]:
# don't worry about this part if running notebook offline. 
# gsl was installed when building binder online, but here LD_LIBRARY_PATH requires to be updated
if 'NB_PYTHON_PREFIX' in os.environ.keys():
    os.environ['LD_LIBRARY_PATH'] = os.environ['NB_PYTHON_PREFIX'] +'/lib'

# 2 Loading Baseline Model & Extracted Event, Parse Event

In [6]:
# To keep root folder intact, load inputs from tutorial directory 'examples/input/'
in_dir = 'input'
isExist = os.path.exists(in_dir)
if not isExist:
    os.makedirs(in_dir)

In [7]:
# support your input baseline model here
model_file = 'BooleanTcell.xlsx'
baseline = os.path.join(in_dir,model_file)

In [8]:
# Load baseline model file into two dictionaries
# we highlight the key information of the baseline model
from runAccordion import get_model
model_dict, regulators = get_model(baseline)
df_model = pd.read_excel(baseline, index_col=0)
df_model[['Element name','Element IDs','Positive regulators','Negative regulators','Levels','Initial 0']]

,Element name,Element IDs,Positive regulators,Negative regulators,Levels,Initial 0
#,,,,,,
1,AKT,P31749,"(PDK1,MTORC2)",AKT_OFF,2,0
2,AKT_OFF,P31749,NaN,NaN,2,0
3,AP1,"P05412,P01100","(FOS_DD,JUN)",NaN,2,0
4,CA,NaN,TCR,NaN,2,0
5,CD122,P14784,NaN,NaN,2,1
...,...,...,...,...,...,...
58,TCR_HIGH,"P01850,P01848",NaN,NaN,2,0
59,TCR_LOW,"P01850,P01848",NaN,NaN,2,0
60,TGFBETA,P01137,NaN,NaN,2,0


In [9]:
# 'model_dict' is a dictionary that holds critical information of each baseline model element
# 'regulators' is a dict that contains baseline model elements and corresponding regulator elements
# model_dict
regulators

{'AKT': {'AKT_OFF', 'MTORC2', 'PDK1'},
 'AKT_OFF': set(),
 'AP1': {'FOS_DD', 'JUN'},
 'CA': {'TCR'},
 'CD122': set(),
 'CD132': set(),
 'CD25': {'AP1', 'FOXP3', 'NFAT', 'NFKAPPAB', 'STAT5'},
 'CD28': set(),
 'ERK': {'MEK2'},
 'FOS': {'ERK'},
 'FOS_D': {'FOS'},
 'FOS_DD': {'FOS_D'},
 'FOS_DDD': set(),
 'FOXP3': {'MTOR_DD', 'NFAT', 'SMAD3', 'STAT5'},
 'IL2': {'AP1', 'FOXP3', 'IL2', 'NFAT', 'NFKAPPAB'},
 'IL2_EX': {'IL2', 'IL2_EX'},
 'IL2R': {'CD122', 'CD132', 'CD25'},
 'JAK3': {'IL2R', 'IL2_EX'},
 'JNK': {'MKK7'},
 'JUN': {'JNK'},
 'MEK2': {'RAF'},
 'MKK7': {'TAK1'},
 'MTOR': {'MTORC1_D', 'MTORC2_D'},
 'MTOR_D': {'MTOR'},
 'MTOR_DD': {'MTOR_D'},
 'MTOR_DDD': set(),
 'MTOR_DDDD': set(),
 'MTORC1': {'MTORC1_OFF', 'RHEB'},
 'MTORC1_D': {'MTORC1'},
 'MTORC1_OFF': set(),
 'MTORC2': {'PI3K_HIGH', 'PI3K_LOW', 'S6K1'},
 'MTORC2_D': {'MTORC2'},
 'MTORC2_DD': set(),
 'NFAT': {'CA', 'NFAT_OFF'},
 'NFAT_OFF': set(),
 'NFKAPPAB': {'AKT', 'PKCTHETA'},
 'PDK1': {'PIP3'},
 'PI3K': {'PI3K_HIGH', 'PI3K_LO

In [10]:
# support your extracted event information here, it is from machine reading
reading_file = 'MachineReadingOutput.csv'
reading = os.path.join(in_dir,reading_file)

In [11]:
# we highlight the key information of each event
df_ro = pd.read_csv(reading)
df_ro[['regulator_name', 'ID1', 'regulated_name','ID2', 'interaction', 'PaperID']]

,regulator_name,ID1,regulated_name,ID2,interaction,PaperID
0,AKT,P31749,CD4,P01730,decreases,PMC2275380
1,AKT,P31749,CTRL,P40313,decreases,PMC2275380
2,TGFBETA,P01137,AKT,P31749,increases,PMC2275380
3,Foxp3,Q9BZS1,Ctla4,P16410,increases,PMC2275380
4,Foxp3,Q9BZS1,Gpr83,Q9NYM4,increases,PMC2275380
...,...,...,...,...,...,...
112,MTORC2,Q6R328,MTORC2,Q6R327,increases,PMC4418530
113,CD28,P10747,MTORC2,Q6R328,increases,PMC4418530
114,IL2_EX,P60568,MTORC2,Q6R329,increases,PMC4418530
115,IL2_R,P01589,MTORC2,Q6R330,increases,PMC4418530


In [12]:
# parse the reading output file (extracted events) into a set() object
# each component in the set has format: (regulator element, regulated element, type of interaction (+/-))
from runAccordion import parseExtension
exttt=parseExtension(model_dict, reading)
exttt

{('AKT', 'CD4_ext', '-'),
 ('AKT', 'CTRL_ext', '-'),
 ('AKT', 'FOXO1', '-'),
 ('AKT', 'FoxO3_ext', '-'),
 ('AKT', 'Itk_ext', '-'),
 ('AKT', 'MAGI1_ext', '-'),
 ('AKT', 'MEK1_ext', '-'),
 ('AKT', 'MTORC2', '-'),
 ('AKT', 'TBK1_ext', '+'),
 ('BAM32_ext', 'PAK1_ext', '+'),
 ('BIM_ext', 'PAK1_ext', '+'),
 ('Bcl2_ext', 'CD3_ext', '+'),
 ('Bcl2l11_ext', 'CD4_ext', '+'),
 ('CD25', 'MTORC2', '+'),
 ('CD28', 'MTORC2', '+'),
 ('CD4_ext', 'IL17A_ext', '-'),
 ('CD5_ext', 'CK2_ext', '+'),
 ('CD5_ext', 'P38_ext', '+'),
 ('CHK1_ext', 'AKT', '+'),
 ('CK2_ext', 'AKT', '+'),
 ('CK2_ext', 'CD4_ext', '+'),
 ('CK2_ext', 'CDC37_ext', '+'),
 ('CK2_ext', 'PTEN', '-'),
 ('CTLA4_ext', 'AKT', '-'),
 ('ERK', 'CK2_ext', '+'),
 ('ERK', 'PAK1_ext', '+'),
 ('ERK', 'S5B_ext', '+'),
 ('FASL_ext', 'FAS_ext', '+'),
 ('FOXO1', 'Foxo3a_ext', '-'),
 ('FOXO1', 'PTEN', '+'),
 ('FOXO3_ext', 'FOXO3_ext', '+'),
 ('FOXP3', 'Ctla4_ext', '+'),
 ('FOXP3', 'Gpr83_ext', '+'),
 ('FOXP3', 'Itk_ext', '+'),
 ('HIF1alpha_ext', 'IL17A_ext',

# 3 Clustering, Generate Extension Candidates, Merging

In [13]:
# To keep root folder intact, generate output in tutorial directory 'examples/output'
out_dir = 'output'

isExist = os.path.exists(out_dir)
if not isExist:
    os.makedirs(out_dir)

In [14]:
# set the inflation parameter of the Markov Clustering Algorithm(MCL)
inflation = 2

In [15]:
# This function prepares the inputs to the markov clustering algorithm (MCL)
# 1. it creates a pickle file for the output clusters with interaction information
# 2. it also returns a modified baseline model (without introducing new nodes)
    
from markovCluster import runMarkovCluster
res, new_base_model = runMarkovCluster(out_dir,exttt,model_dict,inflation)

INFO:root:Running the following command through MCL algorithm:
mcl output/abc_model --abc -I 2 -o output/markov_cluster

[mcl] new tab created
[mcl] pid 51129
 ite -----------------  chaos  time hom(avg,lo,hi) m-ie m-ex i-ex fmv
  1  .................  11.98  0.00 1.01/0.05/1.70 4.57 4.57 4.57  71
  2  .................  19.01  0.00 0.70/0.30/1.05 3.64 2.70 12.35  94
  3  .................  11.78  0.00 0.73/0.32/1.14 1.71 0.44 5.46  94
  4  .................   6.19  0.00 0.87/0.39/1.21 3.05 0.36 1.97  94
  5  .................   1.98  0.00 0.92/0.48/1.35 2.51 0.38 0.76  77
  6  .................   1.07  0.00 0.97/0.53/1.02 1.45 0.59 0.45  29
  7  .................   0.57  0.00 0.98/0.60/1.00 1.15 0.68 0.31   9
  8  .................   0.36  0.00 0.99/0.75/1.03 1.00 0.89 0.27   0
  9  .................   0.16  0.00 1.00/0.88/1.16 0.99 0.97 0.27   0
 10  .................   0.31  0.00 1.00/0.81/1.00 0.99 0.98 0.27   0
 11  .................   0.24  0.00 0.99/0.64/1.00 0.99 0.94 0.25   0


In [16]:
# print clusters detected by MCL
with open(os.path.join(out_dir,'markov_cluster'),'r') as f:
    for line in f:
        print(line)

AKT_OFF	AKT	CHK1_ext	CTLA4_ext	IFNgamma_ext	MEK1_ext	PD1_ext	PDK1	SHIP1_ext	TBK1_ext	TGFBETA	TIL_ext	CD4_ext	Bcl2l11_ext	CTRL_ext	FOXO1	FoxO3_ext	Foxo3a_ext	MAGI1_ext

PIP3	PTEN	TCR	HSC_ext	PI3K_HIGH	PI3K_LOW	NEDD4_ext	TCR_HIGH	PIP3_HIGH	PIP3_LOW	P53_ext

CD25	FOXP3	Ctla4_ext	SMAD3	Gpr83_ext	IL2

MTOR	MTOR_DD	MTORC2_D	MTOR_D	Pak1_ext

CD122	IL2R	CD132	IL2_EX	JAK3

MTORC1_D	MTORC1_OFF	MTORC1	RHEB	TSC

JUN	MKK7	JNK	TAK1

PDL1_ext	CD8_ext	Treg_ext	gp120_ext

MEK2	IQGAP1_ext	RAF	RAS

BAM32_ext	PAK1_ext	BIM_ext	siPAK1_ext

MTORC2	NFKAPPAB	CD28

FOS_DD	FOS	FOS_D

CA	NFAT	NFAT_OFF

Itk_ext	Myc_ext	Th17_ext

CK2_ext	CDC37_ext

PI3K	SHIP_ext

STAT5	STAT5_D

Bcl2_ext	CD3_ext

P38_ext	CD5_ext

SP_ext	CD69_ext

ERK	S5B_ext

FASL_ext	FAS_ext

IL17A_ext	HIF1alpha_ext

S6K1	PS6

PKCTHETA	TCR_LOW

AP1



In [17]:
# show clusters with interaction information added, it's one output of runMarkovCluster()
res

[[1,
  ['AKT', 'MEK1_ext', '-'],
  ['TCR', 'AKT', '-'],
  ['PTEN', 'AKT', '-'],
  ['TIL_ext', 'AKT', '-'],
  ['MTOR', 'TBK1_ext', '+'],
  ['AKT', 'FoxO3_ext', '-'],
  ['SHIP1_ext', 'AKT', '-'],
  ['TCR', 'CD4_ext', '-'],
  ['TGFBETA', 'AKT', '+'],
  ['FOXO1', 'PTEN', '+'],
  ['FOXO1', 'Foxo3a_ext', '-'],
  ['TCR', 'MEK1_ext', '+'],
  ['CTLA4_ext', 'AKT', '-'],
  ['AKT', 'Itk_ext', '-'],
  ['PI3K', 'AKT', '-'],
  ['MTOR', 'AKT', '+'],
  ['TBK1_ext', 'AKT', '-'],
  ['PTEN', 'AKT', '+'],
  ['AKT', 'TBK1_ext', '+'],
  ['AKT', 'CD4_ext', '-'],
  ['PI3K', 'AKT', '+'],
  ['PDK1', 'AKT', '+'],
  ['MTORC2', 'AKT', '+'],
  ['AKT', 'MTORC2', '-'],
  ['CHK1_ext', 'AKT', '+'],
  ['PD1_ext', 'AKT', '-'],
  ['AKT', 'MAGI1_ext', '-'],
  ['TBK1_ext', 'FOXO1', '+'],
  ['IFNgamma_ext', 'AKT', '+'],
  ['TBK1_ext', 'CD4_ext', '-'],
  ['TBK1_ext', 'AKT', '+'],
  ['AKT', 'FOXO1', '-'],
  ['CK2_ext', 'AKT', '+'],
  ['AKT', 'CTRL_ext', '-'],
  ['TBK1_ext', 'CD4_ext', '+'],
  ['Itk_ext', 'CD4_ext', '-'],
  ['CD

In [18]:
# show the modified baseline model (without introducing new nodes), it's the other output of runMarkovCluster()
new_base_model

{'AKT': {'AKT_OFF', 'MTORC2', 'PDK1'},
 'AKT_OFF': set(),
 'AP1': {'FOS_DD', 'JUN'},
 'CA': {'TCR'},
 'CD122': set(),
 'CD132': set(),
 'CD25': {'AP1', 'FOXP3', 'NFAT', 'NFKAPPAB', 'STAT5'},
 'CD28': set(),
 'ERK': {'MEK2'},
 'FOS': {'ERK'},
 'FOS_D': {'FOS'},
 'FOS_DD': {'FOS_D'},
 'FOS_DDD': set(),
 'FOXP3': {'MTOR_DD', 'NFAT', 'SMAD3', 'STAT5'},
 'IL2': {'AP1', 'FOXP3', 'IL2', 'NFAT', 'NFKAPPAB'},
 'IL2_EX': {'IL2', 'IL2_EX'},
 'IL2R': {'CD122', 'CD132', 'CD25'},
 'JAK3': {'IL2R', 'IL2_EX'},
 'JNK': {'MKK7'},
 'JUN': {'JNK'},
 'MEK2': {'RAF'},
 'MKK7': {'TAK1'},
 'MTOR': {'MTORC1_D', 'MTORC2_D'},
 'MTOR_D': {'MTOR'},
 'MTOR_DD': {'MTOR_D'},
 'MTOR_DDD': set(),
 'MTOR_DDDD': set(),
 'MTORC1': {'MTORC1_OFF', 'RHEB'},
 'MTORC1_D': {'MTORC1'},
 'MTORC1_OFF': set(),
 'MTORC2': {'PI3K_HIGH', 'PI3K_LOW', 'S6K1'},
 'MTORC2_D': {'MTORC2'},
 'MTORC2_DD': set(),
 'NFAT': {'CA', 'NFAT_OFF'},
 'NFAT_OFF': set(),
 'NFKAPPAB': {'AKT', 'PKCTHETA'},
 'PDK1': {'PIP3'},
 'PI3K': {'PI3K_HIGH', 'PI3K_LO

In [19]:
return_path_TH = 5

In [20]:
# This function records indices of clusters to be merged based on the existence of return paths
# It generates the grouped_ext_Merged pickle file that contains the merged clusters

from runAccordion import merge_clusters
merge_clusters(regulators, out_dir, return_path_TH)

INFO:root:Merge clusters NO.1 and NO.2


# 4 Extend Model to Create New Spreadsheet

In [21]:
# prepare the merged clusters for new spreadsheet creation
p_file = open(os.path.join(out_dir,'grouped_ext_Merged'),'rb')
merged_res = pickle.load(p_file)

In [22]:
# output extended models are stored in 'examples/output/'' and named with index
original_model = os.path.join(in_dir, model_file)
extended_name = os.path.join(out_dir, model_file.split('.')[0] + '_Extension_')

In [23]:
# choose candidate extension, either from these unmerged clusters or these merged clusters

# merged one 
#candidate = merged_res

# unmerged one
candidate = res

In [24]:
# modify baseline model to obtain multiple candidate extended models
from runAccordion import extend_model
for i in range(len(candidate)):
    extend_model(original_model,candidate[i],extended_name+'Candidate_'+str(i+1)+'.xlsx')

In [25]:
# for example, see the first candidate extended model
df_extended_model = pd.read_excel(extended_name+'Candidate_1.xlsx', index_col=0)
df_extended_model[['Element name','Element IDs','Positive regulators','Negative regulators','Levels','Initial 0']]

,Element name,Element IDs,Positive regulators,Negative regulators,Levels,Initial 0
#,,,,,,
1.0,AKT,P31749,"(PDK1,MTORC2),TGFBETA,MTOR,PTEN,PI3K,PDK1,MTOR...","AKT_OFF,TCR,PTEN,TIL_ext,SHIP1_ext,CTLA4_ext,P...",2.0,0
2.0,AKT_OFF,P31749,NaN,NaN,2.0,0
3.0,AP1,"P05412,P01100","(FOS_DD,JUN)",NaN,2.0,0
4.0,CA,NaN,TCR,NaN,2.0,0
5.0,CD122,P14784,NaN,NaN,2.0,1
...,...,...,...,...,...,...
NaN,IFNgamma_ext,NaN,NaN,NaN,NaN,1
NaN,CK2_ext,NaN,NaN,NaN,NaN,1
NaN,CTRL_ext,NaN,NaN,AKT,NaN,1


In [26]:
# uncomment to see the second candidate extended model
# df_extended_model = pd.read_excel(extended_name+'Candidate_2.xlsx', index_col=0)
# df_extended_model[['Element name','Element IDs','Positive regulators','Negative regulators','Levels','Initial 0']]

# 5 Test Candidate Models against Properties

#### See tutorial notebook of model checking (under https://github.com/pitt-miskov-zivanov-lab) for more detailed usages of model checking feature

In [27]:
import Checking_checking as chk
from Extension_checker_extension import call_checker

In [28]:
# To keep root folder intact, generate MC-related output in tutorial directory 'examples/checking'
checking_dir = 'checking'
isExist = os.path.exists(checking_dir)
if not isExist:
    os.makedirs(checking_dir)

In [29]:
# To keep root folder intact, store simulation traces in tutorial directory 'examples/traces'
# statistical model checking requires model simulation
traces_dir = 'traces'
isExist = os.path.exists(traces_dir)
if not isExist:
    os.makedirs(traces_dir) 

In [30]:
# create a test file from a specified coverage
# these parameters are for Bayesian approach to model checking
# check the following paper for detailed definitions
# https://pzuliani.github.io/papers/hscc2010.pdf

# we often keep alpha=1, beta=1 test='BEST', delta=0.05 and change coverage
# test='BEST', delta=0.05, coverage=0.9, alpha=1, beta=1

test_file = os.path.join(checking_dir,'example_test.txt')

coverage = 0.90
test_str = chk.create_test(coverage)

with open(test_file, 'w') as f:
    f.write(test_str)

test_str

'BEST 0.05 0.9 1 1'

In [31]:
# give directory of property files here
# check dependencies/Model_Checking/dishwrap_v1.0/BLTL_README.md for instructions on composing properties
# you may have different properties for different scenario
# e.g. 'input/TheProperties/p1' stores the properties that should be satisfied under the first scenario
# therefore studied_scenario = '0'
# choose studied_scenario = '1' when accessing properties at input/TheProperties/p2
# choose studied_scenario = '2' when accessing properties at input/TheProperties/p3
property_dir = os.path.join('input', 'TheProperties', 'p1')
property_file_list = os.listdir(property_dir)

studied_scenario = '0'

In [32]:
# parameter N is 'steps' in call_checker function
# it indicates how many steps model simulation should run
# this number should be at least a few more steps than the largest ending timestamp 
# among all your nested properties in above folder, e.g. 1555 (5 more than max(1500+50, 1200+200)=1550)
# for property expressions F[1500]G[50](AKT==0), F[1200]G[200](IL2==0)
N = 1555

In [33]:
# to save time, we only demo model checking below for the first two candidates
# uncomment this cell to test all candidates and make comparisons
candidate = candidate[:2]

In [34]:
# For each model file and each property expression, run model checking to obtain probability.
# When number of candidate models or properties is high, or the property itself is highly nested or complicated,
# running time of this cell could be long

probability_df = pd.DataFrame(columns=property_file_list, 
                              index=['Extension_Candidate_'+str(i+1) for i in range(len(candidate))])

for can_name in probability_df.index:
    estimate_result = []
    
    for p_f in property_file_list:
        
        # suppport candidate model file
        extended = model_file.split('.')[0] + '_' + can_name
        candidate_model_file = os.path.join(out_dir, extended+'.xlsx')
        
        # support property file
        property_file = os.path.join(property_dir, p_f)
        print('Testing {} against {}'.format(can_name, property_file))
        
        # define trace file name and store simulation trajectories
        # statistical model checking requires multiple trajectories
        checker_trace_file = os.path.join(traces_dir, 'example_traces_' + extended + '.txt')
        
        # define output estimate file that contains model checking results
        estimate_file = os.path.join(checking_dir, 'example_estimate_' + extended + '_' + p_f)
        
        # framework_path is intentionally written in Mac/Linux style, not using os.path.join()
        framework_path = '../dependencies/Model_Checking'
        
        # call function to run model checking, method varies with different OS
        if platform.system() == 'Windows' or platform.system().startswith('CYGWIN'):
            f_out = open(estimate_file, 'w')
            command3 = "\"{}/dishwrap_v1.0/dishwrap/dishwrap.exe\" \
            {} \
            {}/dishwrap_v1.0/monitor/checker.exe \
            {} \
            {}/Simulation_Simulator_Python_simulator_interface.py \
            {} \
            {} \
            {} \
            {} \
            {}".format(framework_path,
                       test_file.replace('\\','/'), 
                       framework_path, 
                       property_file.replace('\\','/'),
                       framework_path,
                       candidate_model_file.replace('\\','/'),
                       checker_trace_file.replace('\\','/'),
                       studied_scenario,
                       str(0),
                       str(N)
                      )
            process = subprocess.Popen(command3, shell=True, stdout=f_out)
            process.wait()
        else:
            estimate = call_checker(
                test_file, property_file, candidate_model_file, 
                checker_trace_file, estimate_file, 
                framework_path, scenario=studied_scenario, 
                steps=N)
        
        p = chk.get_estimate(estimate_file)
        print('Probability: ', p)
        
        estimate_result.append(p)
        
    probability_df.loc[can_name] = estimate_result

Testing Extension_Candidate_1 against input/TheProperties/p1/p1d.txt
Probability:  0.794286
Testing Extension_Candidate_1 against input/TheProperties/p1/p1c.txt
Probability:  0.956522
Testing Extension_Candidate_1 against input/TheProperties/p1/p1b.txt
Probability:  0.956522
Testing Extension_Candidate_1 against input/TheProperties/p1/p1a.txt
Probability:  0.956522
Testing Extension_Candidate_2 against input/TheProperties/p1/p1d.txt
Probability:  0.956522
Testing Extension_Candidate_2 against input/TheProperties/p1/p1c.txt
Probability:  0.956522
Testing Extension_Candidate_2 against input/TheProperties/p1/p1b.txt
Probability:  0.956522
Testing Extension_Candidate_2 against input/TheProperties/p1/p1a.txt
Probability:  0.956522


In [35]:
# Show the probability table
probability_df

,p1d.txt,p1c.txt,p1b.txt,p1a.txt
Extension_Candidate_1,0.794286,0.956522,0.956522,0.956522
Extension_Candidate_2,0.956522,0.956522,0.956522,0.956522
